#### 감정 분석 모델 만들기 

In [5]:
#jaffe 레이블 데이터가 잘 열리는지 확인하는 코드 
import os 
import pandas as pd 
import numpy as np 
import scipy 
import matplotlib.pyplot as plt    
%matplotlib inline 

In [6]:
root = './data/jaffe/'
label = 'jaffe_labels.txt'

In [7]:
#데이터 로드 
label_path = os.path.join(root, label)
with open(label_path, 'r') as f:
    print(f.readline())
    print(f.readline())
    for line in f:
        print(line)
        break

# HAP SAD SUR ANG DIS FEA PIC

--------------------------------------------------------------

1 2.87 2.52 2.10 1.97 1.97 2.06 KM-NE1



In [8]:
#텍스트 데이터를 pandas 패키지의 데이터프레임 클래스로 로드하는 코드 
names = ['idx', 'Happy', 'Sad', 'Surprise', 'Angry', 'Disappointed', 'Fear', 'filename']

data = pd.read_csv(label_path, sep=" ", header=1, index_col = None, names = names)

data.head()

,idx,Happy,Sad,Surprise,Angry,Disappointed,Fear,filename
0,1,2.87,2.52,2.10,1.97,1.97,2.06,KM-NE1
1,2,2.87,2.42,1.58,1.84,1.77,1.77,KM-NE2
2,3,2.50,2.10,1.70,1.50,1.73,1.53,KM-NE3
3,4,4.90,1.13,1.26,1.10,1.03,1.10,KM-HA1
4,5,4.87,1.20,1.43,1.03,1.07,1.07,KM-HA2


In [9]:
#데이터프레임에 Neutral 컬럼을 추가하고 값을 0으로 초기화한 다음, 의도에 맞도록 다시 값을 채우는 코드 
#컬럼, 이름, 값 
try:
    data['Neutral']
except KeyError:
    data.insert(1, 'Neutral', 0.)
    
data.loc[data['filename'].str.contains('NE'), 'Neutral'] = 5.0
data.loc[~data['filename'].str.contains('NE'), 'Neutral'] = 1.0

data.head()

,idx,Neutral,Happy,Sad,Surprise,Angry,Disappointed,Fear,filename
0,1,5.0,2.87,2.52,2.10,1.97,1.97,2.06,KM-NE1
1,2,5.0,2.87,2.42,1.58,1.84,1.77,1.77,KM-NE2
2,3,5.0,2.50,2.10,1.70,1.50,1.73,1.53,KM-NE3
3,4,1.0,4.90,1.13,1.26,1.10,1.03,1.10,KM-HA1
4,5,1.0,4.87,1.20,1.43,1.03,1.07,1.07,KM-HA2


In [10]:
#JAFFE 데이터 중 첫 번째 인덱스의 감정 상태를 불러오는 코드 
data.iloc[0][1:8]

Neutral            5
Happy           2.87
Sad             2.52
Surprise         2.1
Angry           1.97
Disappointed    1.97
Fear            2.06
Name: 0, dtype: object

In [11]:
#JAFFE 데이터 중 첫 번째 데이터에서 감정값만 확인하는 코드 
data.iloc[0][['Neutral', 'Happy', 'Sad', 'Surprise', 'Angry', 'Disappointed', 'Fear']]

Neutral            5
Happy           2.87
Sad             2.52
Surprise         2.1
Angry           1.97
Disappointed    1.97
Fear            2.06
Name: 0, dtype: object

In [12]:
#데이터프레임의 컬럼을 명시적으로 재선언하고 데이터의 순서를 바꿔주는 코드 
#컬럼 재배치 

_idx = ['idx', 'Neutral', 'Happy', 'Sad', 'Surprise', 'Angry', 'Disappointed', 'Fear', 'filename']

data = data[_idx]

data.head()

,idx,Neutral,Happy,Sad,Surprise,Angry,Disappointed,Fear,filename
0,1,5.0,2.87,2.52,2.10,1.97,1.97,2.06,KM-NE1
1,2,5.0,2.87,2.42,1.58,1.84,1.77,1.77,KM-NE2
2,3,5.0,2.50,2.10,1.70,1.50,1.73,1.53,KM-NE3
3,4,1.0,4.90,1.13,1.26,1.10,1.03,1.10,KM-HA1
4,5,1.0,4.87,1.20,1.43,1.03,1.07,1.07,KM-HA2


In [13]:
#감정 문자열을 숫자로 맵핑하고 데이터프레임에 label 컬럼 추가 
#감정 문자열을 숫자로 맵핑하기 
label = {'Neutral' : 0,
         'Happy' : 1,
         'Sad' : 2,
         'Surprise' : 3,
         'Angry' : 4,
         'Disappointed' : 5,
         'Fear' : 6}

#파일 이름이 다음과 같은 문자열을 포함한다면 숫자로 맵핑 
f_label = {'NE' : 0,
         'HA' : 1,
         'SA' : 2,
         'SU' : 3,
         'AN' : 4,
         'DI' : 5,
         'FE' : 6}
try:
    data['label']
except KeyError:
    data.insert(9, 'label', np.nan)

data.head()

,idx,Neutral,Happy,Sad,Surprise,Angry,Disappointed,Fear,filename,label
0,1,5.0,2.87,2.52,2.10,1.97,1.97,2.06,KM-NE1,NaN
1,2,5.0,2.87,2.42,1.58,1.84,1.77,1.77,KM-NE2,NaN
2,3,5.0,2.50,2.10,1.70,1.50,1.73,1.53,KM-NE3,NaN
3,4,1.0,4.90,1.13,1.26,1.10,1.03,1.10,KM-HA1,NaN
4,5,1.0,4.87,1.20,1.43,1.03,1.07,1.07,KM-HA2,NaN


In [15]:
#피쳐를 대표하는 감정을 찾고, 그 값이 파일 이름에 명시된 감정과 대치될 경우 파일 이름을 따라서 감정값을 기록하는 코드 
for idx, item in data.iterrows():
    max_label = item[item == item[1:8].max()].index[0]
    max_value = label.get(max_label)
    orig_label = item[-2][-3:-1]
    orig_value = f_label.get(orig_label)
    
    if max_value == orig_value:
        data.at[idx, 'label'] = max_value
    else:
        data.at[idx, 'label'] = orig_value

In [16]:
#sklearn 패키지를 사용해서 학습용 데이터셋과 테스용 데이터셋을 분리하는 코드 
from sklearn.model_selection import train_test_split

#id 제외 
X = data.loc[:, 'Neutral':'filename']
y = data.loc[:, 'label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= 42)

In [18]:
#이미지 데이터의 이름을 입력하면 이미지 파일을 열고 numpy 행렬로 데이터를 불러오는 코드 
import cv2 
import matplotlib

ModuleNotFoundError: No module named 'cv2'

In [19]:
!pip install opencv-python

  Using cached opencv-python-4.9.0.80.tar.gz (92.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done